In [133]:
import sqlite3
import pandas as pd
import numpy as np
import os

'''
The big problem is that all data is stored as bytes, and thus need to be transformed back to actual information.
The float64 multiarrays are decoded into lists with a handy funvtion from numpy. But returns a list. This list
is split into multiple section later in the script
'''
#parses UInt32 msgs
def uint32(byt):
    l = 4
    return int.from_bytes(byt[-l:],"little")

#Parses my float64 WQ data
def wq_float64(byt):
     return np.frombuffer(byt[4:],dtype = np.float64)[2:]
    
#Parses the Float32 motors
def float32_motors(byt):
    
    return np.frombuffer(byt, dtype = np.float32)[-2:]

# Parses generic float64 arrays
def float64(byt):
    
    return np.frombuffer(byt[4:], dtype = np.float64)[2:]

# Parses GPS data

def gps_float64(byt):
    d = np.frombuffer(byt[20:-4],dtype = np.float64)[:3]
    if d.size <3:
        d = [0,0,0] # to stop erroring for no fix gps 0.0.0
    return d

# topics = topics df
# msgs = msgs df
# topic = topic to be selected 
# parser = one of the above
# cols = columns to split the data into, basically the order in which ROS is pulbishing them 
def parse_df(topics, msgs, topic, parser, cols = ["volts", "amps"]):
    #find the specific topic_id that the topic has gained in the bag
    t_id = topics.id[topics["name"].str.contains(topic)]
    #subselect this data out of all the msgs
    df1 = msgs.loc[msgs["topic_id"].isin([t_id])].reset_index()
    #apply the conversion from bytes to actual numbers
    df1[topic] = df1["data"].apply(parser)
    #Create different columns from the lists created by the parser
    df = pd.DataFrame(df1[topic].to_list(), columns = cols)
    #add timestamp again
    df["timestamp"] = df1["timestamp"]
    return df


#name of the ROSbag
rosbagname = "PowerTest7"

#location of the rosbag .db3 file, in my case db_loc = ~/bags/Powertest7/Powertest7_0.db3

db_loc = os.path.join(os.path.expanduser('~'), "bags",rosbagname,rosbagname+"_0.db3")

#create connection to rosbag
con = sqlite3.connect(db_loc)

#read the topics in the rosbag
topics = pd.read_sql("SELECT * from topics", con)
# read all the msgs in the rosbag
msgs = pd.read_sql("SELECT * from messages", con)

#/topics that you want
t1 = "motorR" 
t2 =  "motorL"
t3 =  "box_mon"
t4 = "speed_percent"
t5 = "joy"

#the above function is called to create df out of the rosbag
df_r = parse_df(topics,msgs, t1, float32_motors,cols = ["r_volts", "r_amps"])
df_l = parse_df(topics,msgs, t2, float32_motors,cols = ["l_volts", "l_amps"])
df_box = parse_df(topics,msgs, t3, float32_motors,cols = ["volts", "amps"])
df_speed = parse_df(topics,msgs, t4, float64, cols = ["left", "right"])

#The data is merged to the closest neighbor, based on timestamp.
df = pd.merge_asof(df_r,df_l, on = "timestamp")
df = pd.merge_asof(df,df_box, on = "timestamp")
df = pd.merge_asof(df,df_speed, on = "timestamp")

#select until row n
# n = 150
#df = df[df["timestamp"] < df["timestamp"][n]]


In [126]:
#plot it
import plotly.express as px
fig = px.scatter(x=abs(df["l_amps"]), y= abs(df["r_amps"]))
fig.show()

In [132]:
###### import plotly.express as px
import plotly.subplots as ps
import plotly.graph_objects as go

fig = ps.make_subplots(rows=3, cols=1)
fig.add_trace(go.Scatter(x=df["timestamp"], y= df["left"]),row=1, col=1)
fig.add_trace(go.Scatter(x=df["timestamp"], y= df["right"]),row=1, col=1)
fig.add_trace(go.Scatter(x=df["timestamp"], y= df["r_amps"]),row=2, col=1)
fig.add_trace(go.Scatter(x=df["timestamp"], y= df["l_amps"]),row=2, col=1)
fig.add_trace(go.Scatter(x=df["timestamp"], y= abs(df["amps"])),row=3, col=1)


In [131]:
df = df[df["timestamp"] < df["timestamp"][150]]
